In [1]:
import os
from PIL import  Image

import pandas as pd
import seaborn as sns#visualization
import itertools
import warnings
import io
import plotly.offline as py#visualization
py.init_notebook_mode(connected=True)#visualization
import plotly.graph_objs as go#visualization
import plotly.tools as tls#visualization
import plotly.figure_factory as ff#visualization

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix,accuracy_score,classification_report
from sklearn.metrics import roc_auc_score,roc_curve,scorer
from sklearn.metrics import f1_score
import statsmodels.api as sm
from sklearn.metrics import precision_score,recall_score

from sklearn.ensemble import GradientBoostingClassifier

In [2]:
%matplotlib inline
warnings.filterwarnings("ignore")

### Task_1:

1.	повторить анализ набора данных WA_Fn-UseC_-Telco-Customer-Churn.csv (желательно придумать какие-то дополнительные признаки)

In [3]:
telcom = pd.read_csv(r"./WA_Fn-UseC_-Telco-Customer-Churn.csv")
telcom.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [4]:
telcom.shape

(7043, 21)

In [5]:
telcom.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 21 columns):
customerID          7043 non-null object
gender              7043 non-null object
SeniorCitizen       7043 non-null int64
Partner             7043 non-null object
Dependents          7043 non-null object
tenure              7043 non-null int64
PhoneService        7043 non-null object
MultipleLines       7043 non-null object
InternetService     7043 non-null object
OnlineSecurity      7043 non-null object
OnlineBackup        7043 non-null object
DeviceProtection    7043 non-null object
TechSupport         7043 non-null object
StreamingTV         7043 non-null object
StreamingMovies     7043 non-null object
Contract            7043 non-null object
PaperlessBilling    7043 non-null object
PaymentMethod       7043 non-null object
MonthlyCharges      7043 non-null float64
TotalCharges        7043 non-null object
Churn               7043 non-null object
dtypes: float64(1), int64(2), obj

In [6]:
telcom.describe()

,SeniorCitizen,tenure,MonthlyCharges
count,7043.000000,7043.000000,7043.000000
mean,0.162147,32.371149,64.761692
std,0.368612,24.559481,30.090047
min,0.000000,0.000000,18.250000
25%,0.000000,9.000000,35.500000
50%,0.000000,29.000000,70.350000
75%,0.000000,55.000000,89.850000
max,1.000000,72.000000,118.750000


In [7]:
telcom.corr()

,SeniorCitizen,tenure,MonthlyCharges
SeniorCitizen,1.000000,0.016567,0.220173
tenure,0.016567,1.000000,0.247900
MonthlyCharges,0.220173,0.247900,1.000000


In [8]:
telcom.isna().sum()

customerID          0
gender              0
SeniorCitizen       0
Partner             0
Dependents          0
tenure              0
PhoneService        0
MultipleLines       0
InternetService     0
OnlineSecurity      0
OnlineBackup        0
DeviceProtection    0
TechSupport         0
StreamingTV         0
StreamingMovies     0
Contract            0
PaperlessBilling    0
PaymentMethod       0
MonthlyCharges      0
TotalCharges        0
Churn               0
dtype: int64

In [9]:
telcom.nunique()

customerID          7043
gender                 2
SeniorCitizen          2
Partner                2
Dependents             2
tenure                73
PhoneService           2
MultipleLines          3
InternetService        3
OnlineSecurity         3
OnlineBackup           3
DeviceProtection       3
TechSupport            3
StreamingTV            3
StreamingMovies        3
Contract               3
PaperlessBilling       2
PaymentMethod          4
MonthlyCharges      1585
TotalCharges        6531
Churn                  2
dtype: int64

In [10]:
#Data Manipulation

#Replacing spaces with null values in total charges column
telcom['TotalCharges'] = telcom["TotalCharges"].replace(" ",np.nan)

#Dropping null values from total charges column which contain .15% missing data 
telcom = telcom[telcom["TotalCharges"].notnull()]
telcom = telcom.reset_index()[telcom.columns]

#convert to float type
telcom["TotalCharges"] = telcom["TotalCharges"].astype(float)

#replace 'No internet service' to No for the following columns
replace_cols = [ 'OnlineSecurity', 'OnlineBackup', 'DeviceProtection',
                'TechSupport','StreamingTV', 'StreamingMovies']
for i in replace_cols : 
    telcom[i]  = telcom[i].replace({'No internet service' : 'No'})
    
#replace values
telcom["SeniorCitizen"] = telcom["SeniorCitizen"].replace({1:"Yes",0:"No"})

#Tenure to categorical column
def tenure_lab(telcom) :
    
    if telcom["tenure"] <= 12 :
        return "Tenure_0-12"
    elif (telcom["tenure"] > 12) & (telcom["tenure"] <= 24 ):
        return "Tenure_12-24"
    elif (telcom["tenure"] > 24) & (telcom["tenure"] <= 48) :
        return "Tenure_24-48"
    elif (telcom["tenure"] > 48) & (telcom["tenure"] <= 60) :
        return "Tenure_48-60"
    elif telcom["tenure"] > 60 :
        return "Tenure_gt_60"
telcom["tenure_group"] = telcom.apply(lambda telcom:tenure_lab(telcom),
                                      axis = 1)

#Separating churn and non churn customers
churn     = telcom[telcom["Churn"] == "Yes"]
not_churn = telcom[telcom["Churn"] == "No"]

#Separating catagorical and numerical columns
Id_col     = ['customerID']
target_col = ["Churn"]
cat_cols   = telcom.nunique()[telcom.nunique() < 6].keys().tolist()
cat_cols   = [x for x in cat_cols if x not in target_col]
num_cols   = [x for x in telcom.columns if x not in cat_cols + target_col + Id_col]

In [11]:
#cusomer attrition in tenure groups
tg_ch  =  churn["tenure_group"].value_counts().reset_index()
tg_ch.columns  = ["tenure_group","count"]
tg_nch =  not_churn["tenure_group"].value_counts().reset_index()
tg_nch.columns = ["tenure_group","count"]

#bar - churn
trace1 = go.Bar(x = tg_ch["tenure_group"]  , y = tg_ch["count"],
                name = "Churn Customers",
                marker = dict(line = dict(width = .5,color = "black")),
                opacity = .9)

#bar - not churn
trace2 = go.Bar(x = tg_nch["tenure_group"] , y = tg_nch["count"],
                name = "Non Churn Customers",
                marker = dict(line = dict(width = .5,color = "black")),
                opacity = .9)

layout = go.Layout(dict(title = "Customer attrition in tenure groups",
                        plot_bgcolor  = "rgb(243,243,243)",
                        paper_bgcolor = "rgb(243,243,243)",
                        xaxis = dict(gridcolor = 'rgb(255, 255, 255)',
                                     title = "tenure group",
                                     zerolinewidth=1,ticklen=5,gridwidth=2),
                        yaxis = dict(gridcolor = 'rgb(255, 255, 255)',
                                     title = "count",
                                     zerolinewidth=1,ticklen=5,gridwidth=2),
                       )
                  )
data = [trace1,trace2]
fig  = go.Figure(data=data,layout=layout)
py.iplot(fig)

In [12]:
tel_df = telcom.copy()

In [13]:
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler

#customer id col
Id_col     = ['customerID']
#Target columns
target_col = ["Churn"]
#categorical columns
cat_cols   = telcom.nunique()[telcom.nunique() < 6].keys().tolist()
cat_cols   = [x for x in cat_cols if x not in target_col]
#numerical columns
num_cols   = [x for x in telcom.columns if x not in cat_cols + target_col + Id_col]
#Binary columns with 2 values
bin_cols   = telcom.nunique()[telcom.nunique() == 2].keys().tolist()
#Columns more than 2 values
multi_cols = [i for i in cat_cols if i not in bin_cols]

#Label encoding Binary columns
le = LabelEncoder()
for i in bin_cols :
    telcom[i] = le.fit_transform(telcom[i])
    
#Duplicating columns for multi value columns
telcom = pd.get_dummies(data = telcom,columns = multi_cols )

#Scaling Numerical columns
std = StandardScaler()
scaled = std.fit_transform(telcom[num_cols])
scaled = pd.DataFrame(scaled,columns=num_cols)

#dropping original values merging scaled values for numerical columns
df_telcom_og = telcom.copy()
telcom = telcom.drop(columns = num_cols,axis = 1)
telcom = telcom.merge(scaled,left_index=True,right_index=True,how = "left")

In [14]:
telcom.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,PhoneService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,...,PaymentMethod_Electronic check,PaymentMethod_Mailed check,tenure_group_Tenure_0-12,tenure_group_Tenure_12-24,tenure_group_Tenure_24-48,tenure_group_Tenure_48-60,tenure_group_Tenure_gt_60,tenure,MonthlyCharges,TotalCharges
0,7590-VHVEG,0,0,1,0,0,0,1,0,0,...,1,0,1,0,0,0,0,-1.280248,-1.161694,-0.994194
1,5575-GNVDE,1,0,0,0,1,1,0,1,0,...,0,1,0,0,1,0,0,0.064303,-0.260878,-0.173740
2,3668-QPYBK,1,0,0,0,1,1,1,0,0,...,0,1,1,0,0,0,0,-1.239504,-0.363923,-0.959649
3,7795-CFOCW,1,0,0,0,0,1,0,1,1,...,0,0,0,0,1,0,0,0.512486,-0.747850,-0.195248
4,9237-HQITU,0,0,0,0,1,0,0,0,0,...,1,0,1,0,0,0,0,-1.239504,0.196178,-0.940457


In [15]:
telcom.drop(columns=['customerID'], inplace=True)

In [16]:
cols = [i for i in telcom.columns if i not in Id_col + target_col]

##### Добавляем новые признаки:

In [17]:
# Кол-во подключенных сервисов.
telcom['total_services'] = telcom['PhoneService'] + telcom['OnlineSecurity'] + telcom['OnlineBackup'] +\
telcom['DeviceProtection'] + telcom['TechSupport'] + telcom['StreamingTV'] + telcom['StreamingMovies']

# Подключены ли все стриминговые сервисы.
telcom['all_streaming'] = 0
telcom.loc[(telcom['StreamingTV'] == 1) & (telcom['StreamingMovies'] == 1), 'all_streaming'] = 1

# Подключены ли все сервисы безопасности.
telcom['all_security'] = 0
telcom.loc[(telcom['OnlineSecurity'] == 1) & (telcom['OnlineBackup'] == 1) & (telcom['DeviceProtection'] == 1),
           'all_security'] = 1

##### Делим на обучающую и отложенную выборки:

In [18]:
train,test = train_test_split(telcom,test_size = .25 ,random_state = 111)
    
##seperating dependent and independent variables
cols    = [i for i in telcom.columns if i not in Id_col + target_col]
train_X = train[cols]
train_Y = train[target_col]
test_X  = test[cols]
test_Y  = test[target_col]

### Task_2:

2.	дополнить список моделей еще как минимум тремя (lightgbm, xgboost, svm, knn, naivebayes, сети, что-то еще)

In [19]:
def calc_metrics(name, predict, predicted_proba):
    accuracy     = accuracy_score(test_Y, predict)
    recallscore  = recall_score(test_Y, predict)
    precision    = precision_score(test_Y, predict)
    roc_auc      = roc_auc_score(test_Y, predicted_proba)
    f1score      = f1_score(test_Y, predict) 
    
    results_ = pd.DataFrame({"Model"      : [name],
                       "Accuracy_score"  : [accuracy],
                       "Recall_score"    : [recallscore],
                       "Precision"       : [precision],
                       "f1_score"        : [f1score],
                       "Area_under_curve": [roc_auc]
                      })
    return results_

In [20]:
model = GradientBoostingClassifier()
fitted_model = model.fit(train_X, train_Y)

print('Порядок классов', fitted_model.classes_)

predict = fitted_model.predict(test_X)
predict_proba = fitted_model.predict_proba(test_X)[:, 1]

results = calc_metrics('GradientBoostingClassifier', predict, predict_proba)

f1 = f1_score(test_Y, predict)
roc_auc = roc_auc_score(test_Y, predict_proba)
f1, roc_auc

Порядок классов [0 1]


(0.5995316159250585, 0.8477917981072556)

In [21]:
from lightgbm import LGBMClassifier

In [22]:
params = {}
params ['learning_rate'] = 0.003
params ['boosting_type'] = 'gbdt'
params ['target'] = 'binary'
params ['metric'] = 'binary_logloss'
params ['sub_feature'] = 0.5
params ['num_leaves'] = 10
params ['min_data'] = 50
params ['max_depth'] = 10

model = LGBMClassifier()
model = model.fit(train_X, train_Y)
predict = model.predict(test_X)
predicted_proba = model.predict_proba(test_X)[:,1]

results = pd.concat([results, calc_metrics('LGBMClassifier', predict, predicted_proba)])

roc_auc = roc_auc_score(test_Y, predicted_proba)
roc_auc

0.8372964012103264

In [23]:
import xgboost as xgb

In [24]:
params = {'max_depth':2, 'eta':1, 'objective':'binary:logistic' }


num_round = 2

bst = xgb.XGBClassifier()
bst = bst.fit(train_X, train_Y)

print('Порядок классов', bst.classes_)

predicted_proba = bst.predict_proba(test_X)[:, 1]
predict = bst.predict(test_X)

results = pd.concat([results, calc_metrics('XGBClassifier', predict, predicted_proba)])

roc_auc = roc_auc_score(test_Y, predicted_proba)
roc_auc

Порядок классов [0 1]


0.8474216184896672

In [25]:
from sklearn.neighbors import KNeighborsClassifier

In [26]:
neigh = KNeighborsClassifier(n_neighbors=25)
neigh = neigh.fit(train_X, train_Y)

print(neigh.classes_)

predict = neigh.predict(test_X)
predicted_proba = neigh.predict_proba(test_X)[:,1]

results = pd.concat([results, calc_metrics('KNeighborsClassifier', predict, predicted_proba)])

# f1 = f1_score(test_Y, predict)
roc_auc = roc_auc_score(test_Y, predicted_proba)
# f1, roc_auc
roc_auc

[0 1]


0.8377148651258611

### Task_3:

3.	на выходе сформировать единую таблицу метрик качества со всеми моделями

In [27]:
results.reset_index(drop=True)

,Model,Accuracy_score,Recall_score,Precision,f1_score,Area_under_curve
0,GradientBoostingClassifier,0.805461,0.522449,0.703297,0.599532,0.847792
1,LGBMClassifier,0.787827,0.508163,0.653543,0.571757,0.837296
2,XGBClassifier,0.800341,0.504082,0.695775,0.584615,0.847422
3,KNeighborsClassifier,0.789534,0.530612,0.650000,0.584270,0.837715


### Task_4:

4.	сделать выводы о том, какие модели лучше справляются с задачей прогноза оттока на этой выборке

##### Вывод:

По большинству метрик ТОП2 моделей по качеству: GradientBoostingClassifier, XGBClassifier. Однако, все модели брались с какими-то стандартными значениями гипер-параметров, либо не стандартными, но без подбора оптимальных параметров. Таким образом данном распределение мест может не отражать реальную способность конкретного типа моделей с точки зрения максимального качества предиктов на предложенных данных, а отражают только качество конкретных моделей.

### Task_5:

5.	есть ли смысл в этапе отборе признаков с точки зрения повышения качества прогнозов

Отбор признаков имеет в себе две составляющие (ожидаемые последствия) - уменьшение кол-ва признков и увеличение среднего "качества" признака в признаковом пространстве. Уменьшение кол-ва признаков практически однозначно положительно сказывается на сложности модели, и положительно сказываетя на всем, что на это завязано - ресурсо-затратность и скорость вычислений и т.д. Повышение же среднего качества признака в наборе признаков может положительно сказаться при использовании некоторых типов моделей, а так же при относительно небольшом объеме обучающей выборки, в большинстве же остальных случаев внутренняя логика работы моделей предполагает внутренние механизмы отбора признаков, таким образом, большее влияние на прогнозы будут оказывать как раз более значимые признаки. В то же время уменьшения кол-ва признаков, пусть даже за счет наименее значимых, это всегда потеря информации и это может привести к некотрому ухудшению качества моделей. В любом случае в каждом конкретном случае целесообразность отбора признаков по качеству - это скорее вопрос исследования - в смысле надо посмотреть и оценить для конкретного кейса.

### Task_6:

6.	(опционально) сделать подбор параметров (без особого усердия) для лучшей (можно нескольких) моделей и проверить качество снова

In [79]:
from random import choice

max_depth_ = [1, 2, 3, 4, 5]
learning_rate_ = [0.01, 0.025, 0.05, 0.1, 0.15, 0.2, 0.25, 0.5, 0.75, 1]
n_estimators_ = [10, 25, 50, 100, 250, 500, 750, 1000]

results_ = {}

for i in range(10):
    max_depth = choice(max_depth_)
    learning_rate = choice(learning_rate_)
    n_estimators = choice(n_estimators_)
    
    bst = xgb.XGBClassifier(max_depth=max_depth, learning_rate=learning_rate, n_estimators=n_estimators,
                              silent=True, objective='binary:logistic', booster='gbtree')
    bst = bst.fit(train_X, train_Y)
    predicted_proba = bst.predict_proba(test_X)[:, 1]
    roc_auc = round(roc_auc_score(test_Y, predicted_proba), 4)
    results_[f'{max_depth}_{learning_rate}_{n_estimators}'] = roc_auc
    print(f'{i} - max_depth: {max_depth}, learning_rate: {learning_rate}, n_estimators: {n_estimators}: {roc_auc}')

max_depth: 4, learning_rate: 0.15, n_estimators: 50: 0.8466
max_depth: 3, learning_rate: 0.5, n_estimators: 25: 0.8427
max_depth: 1, learning_rate: 0.15, n_estimators: 100: 0.8507
max_depth: 4, learning_rate: 1, n_estimators: 50: 0.8111
max_depth: 3, learning_rate: 0.75, n_estimators: 100: 0.8195
max_depth: 2, learning_rate: 0.01, n_estimators: 25: 0.7942
max_depth: 4, learning_rate: 0.1, n_estimators: 500: 0.8273
max_depth: 5, learning_rate: 0.25, n_estimators: 1000: 0.8026
max_depth: 2, learning_rate: 1, n_estimators: 250: 0.8126
max_depth: 5, learning_rate: 0.05, n_estimators: 100: 0.8487
max_depth: 3, learning_rate: 0.025, n_estimators: 100: 0.8478
max_depth: 3, learning_rate: 1, n_estimators: 25: 0.8373
max_depth: 1, learning_rate: 0.1, n_estimators: 25: 0.8316
max_depth: 2, learning_rate: 0.2, n_estimators: 250: 0.8423
max_depth: 2, learning_rate: 0.1, n_estimators: 1000: 0.8363
max_depth: 5, learning_rate: 1, n_estimators: 1000: 0.7842
max_depth: 1, learning_rate: 0.5, n_estimat

KeyboardInterrupt: 

In [81]:
a = list(results_.items())
a.sort(key=lambda param: param[1], reverse=True)

a[:10]

[('1_0.25_100', 0.8526),
 ('1_0.1_250', 0.8517),
 ('1_0.5_50', 0.8516),
 ('1_0.1_750', 0.8511),
 ('1_0.15_500', 0.8508),
 ('1_0.15_100', 0.8507),
 ('1_0.25_500', 0.8504),
 ('3_0.05_100', 0.85),
 ('2_0.25_50', 0.85),
 ('1_0.5_250', 0.8499)]

In [82]:
max_depth = 1
learning_rate = 0.1
n_estimators = 250

bst = xgb.XGBClassifier(max_depth=max_depth, learning_rate=learning_rate, n_estimators=n_estimators,
                          silent=True, objective='binary:logistic', booster='gbtree')
bst = bst.fit(train_X, train_Y)
predicted_proba = bst.predict_proba(test_X)[:, 1]
predict = bst.predict(test_X)

results = calc_metrics('KNeighborsClassifier', predict, predicted_proba)
results

,Model,Accuracy_score,Recall_score,Precision,f1_score,Area_under_curve
0,KNeighborsClassifier,0.803185,0.510204,0.702247,0.591017,0.851704


Удалось улучшить модель по всем метрикам, но незначительно.